### Closure(闭包)

## 变量作用域

### 函数对象的作用域

python中一切皆对象，同其他对象一样,函数对象也有其使用的范围即函数对象的作用域。

在python中我们通过def定义函数，函数对象的作用域与def所在的层级相同

## 闭包

关于闭包主要有下面两种说法：

* 闭包是符合一定条件的函数，定义为：闭包是在其词法上下文中引用了自由变量的函数
* 闭包是由函数与其相关的引用环境组合而成的实体。定义为：在实现绑定时，需要创建一个能显示表示引用环境的东西，并将它与相关的子程序捆绑在一起，这样捆绑起来的整体称为闭包

个人觉得第二种说法更准确，闭包只是在形式上表现像函数，实际不是函数。

函数的定义是：一些可执行的代码，这些代码在函数定义后就确定了，不会在执行时发生变化，所以一个函数只有一个实例。

闭包在运行的时候可以有多个实例，不同的引用环境和相同的环境组合可以产生不同的实例。

引用环境: 其实就是在执行运行的某个时间点，所有处于活跃状态的变量组成的集合，这里的变量是指变量的名字和其所代表的对象之间的联系。

可以使用闭包语言的特点：

* 函数可以作为另外一个函数的返回值或者参数，还可以作为一个变量的值。
* 函数可以嵌套使用

而认为闭包是函数的有一句话是：闭包是指延伸了作用域的函数，其中包含函数定义体中引用。但是不在定义体中定义的非全局变量。个人觉得也是一种理解方式

通过下面例子更好理解：

先实现一种计算平均值的方法：

In [1]:
class Average(object):
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Average()
print(avg(10))
print(avg(20))
print(avg(30))

10.0
15.0
20.0


### 使用闭包

In [4]:
def make_average():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_average()
print(avg(10))
print(avg(20))
print(avg(30))
print(dir(avg))
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

10.0
15.0
20.0
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
('new_value', 'total')
('series',)
(<cell at 0x000001B73873DD68: list object at 0x000001B7386DE308>,)
[10, 20, 30]


通常会认为我们调用avg(10)的时候make_averager函数已经返回了，而它的本地作用域也一去不复返，但这里其实series是自由变量，是指未在本地作用域绑定的变量, 我们可以通过print(dir(avg)),看到结果.

series的绑定在返回的avg函数的`__closure__`属性中, 这或许就是有的人会认为闭包一种函数。闭包会保留定义函数时存在的自由变量的绑定，这样调用函数时虽然定义作用域不能用了，但是仍能使用那些绑定

### nonlocal关键字

刚开始了解闭包之后，如果尝试使用这种编程方式容易出现以下错误使用例子：

In [5]:
def make_average():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_average()
print(avg(10))

UnboundLocalError: local variable 'count' referenced before assignment

这个例子中和上面使用的不同之处是：这里的count和total是数字，是不可变类型，而之前的例子中series是一个列表是可变类型

所以这里回到了最开始说的作用域问题了，当我们在averager中使用`count += 1`的时候, 其实就是`count = count + 1`,这样就是在averager函数定义体中对count进行赋值，count就变成了局部变量。

问题小结：当变量为数字，字符串，元组等不可变类型时，只能读取不能更新，如果使用类似`count += 1`就会隐式的把count变成局部变量，所以开始例子中使用series，我们后面的操作是append并且列表还是可变对象.

不过python3引入了一个新的关键词`nonlocal`，通过它**把变量标记为自由变量**，这样我们把上面这个错误的例子简单更改：

In [6]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count,total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
print(avg(10))
print(avg(20))


10.0
15.0
